<a href="https://colab.research.google.com/github/ma2sevich222/raw_stock_data_researches/blob/main/wvap%20patterns%20matching%20using%20matrix%20profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install stumpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 6.4 MB/s 


In [ ]:
import plotly.graph_objects as go
import io
from google.colab import files
import pandas as pd
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import stumpy
import matplotlib.pyplot as plt

ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in CUDA_ERROR_NO_DEVICE


In [ ]:
uploaded = files.upload()

Saving LABU1-Minute-Trade.csv to LABU1-Minute-Trade.csv


In [ ]:
def add_VWAP_bands(df):
  
   df['Date_Time'] = pd.to_datetime(df['Date_Time'])
   df.set_index('Date_Time', inplace=True)
   df_VWAP = None
   itr = 0
   for idx, day in df.groupby(df.index.date):
       
       day["TP"] = (day["Open"] + day["High"] + day["Low"]+ day["Close"] ) / 4
       day["VTP"] =  day["TP"] * day["TotalVolume"] 
       day["cumVTP"] = day["VTP"].cumsum()
       day["cumV"] = day["TotalVolume"].cumsum()
       day["WVAP"] = day["cumVTP"] / day["cumV"]
       day["Var"] = (((day["TP"] - day["WVAP"])**2) * day["TotalVolume"]).cumsum()
       day["Off"] = (day["Var"]/ day["cumV"])**0.5
       day["+band"] = day["WVAP"] +  day["Off"]
       day["-band"] = day["WVAP"] -  day["Off"]

       if itr == 0:
         df_VWAP = day
         itr +=1
       else:
         df_VWAP = df_VWAP.append(day, ignore_index=True)
   del df_VWAP["TP"],  df_VWAP["VTP"], df_VWAP["cumVTP"], df_VWAP["cumV"],  df_VWAP["Var"], df_VWAP["Off"]
   df_VWAP['Date_Time'] = df.index.values
   df = df.reset_index()
   

   return df_VWAP

In [ ]:
def get_patterns(df, p_size):
  state = 0
  scaler = MinMaxScaler()
  df['Date_Time'] = pd.to_datetime(df['Date_Time'])
  df.set_index('Date_Time', inplace=True)
  patterns =pd.DataFrame()
  for idx, day in df.groupby(df.index.date):
           for i,bar in enumerate(day.index):

             count_day = day.reset_index(drop=True)
         
  
             if day["Close"].loc[bar] >= day["+band"].loc[bar]: 
                state = 1

             if day["Close"].loc[bar] <= day["-band"].loc[bar]:
                state = -1

             if day["Close"].loc[bar] < day["+band"].loc[bar] and state == 1:
                pattern_end = i
                state = 2

             if day["Close"].loc[bar] >day["-band"].loc[bar] and state == -1:
                pattern_end = i
                state = - 2

             if day["Close"].loc[bar] <= day["WVAP"].loc[bar] and state == 2:
                if i - p_size  >= 30 :
                  #patterns.append(scaler.fit_transform(count_day.loc[i - (p_size - 1) : i,["Open", "High","Low","Close","TotalVolume"]].values).flatten())
                  #patterns.append(scaler.fit_transform(count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]].values).flatten())
                  if df.empty:
                    patterns = count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]]
                  else:
                    patterns = patterns.append(count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]], ignore_index=True)

                 
                state = 0
                pattern_len = 0
                pattern_end = 0

             if day["Close"].loc[bar] >= day["WVAP"].loc[bar] and state == -2:
                if i - p_size  >= 30 :
                  #patterns.append(scaler.fit_transform(count_day.loc[i - (p_size - 1) : i,["Open", "High","Low","Close","TotalVolume"]].values).flatten())
                  #patterns.append(scaler.fit_transform(count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]].values).flatten())
                  if df.empty:
                    patterns = count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]]
                  else:
                    patterns = patterns.append(count_day.loc[i - (p_size - 1) : i,["Close","TotalVolume","WVAP","+band","-band"]], ignore_index=True)                  
                state = 0
                pattern_len = 0
                pattern_end = 0
  df = df.reset_index()        
  return patterns

In [ ]:
def plot_predictions(test_df, scores, distance):

  #vwap_df = add_VWAP_bands(test_df)
  patern_index =np.where(scores < distance)
  patern_index=np.array(patern_index[0])
  #patern_index = predictions

  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_df.index, y=test_df["Close"],
                    mode='lines',
                    name='close'),
              )

  fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df['WVAP'],
    mode='lines',
    name='VWAP', 
    line=dict(color='coral',width=2)
))
  fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["+band"],
    mode='lines',
    name='VWAP + 1std', 
    line=dict(color='green',width=2)
))

  fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["-band"],
    mode='lines',
    name='VWAP - 1std', 
    line=dict(color='green',width=2)

))


  fig.add_trace(go.Bar(
    x=patern_index,
    y=[test_df["Close"].max() for i in range(patern_index.shape[0])],
    name='pattern',
    marker_color='black',opacity=1
))
  fig.update_layout(
    title="Разметка паттернов методом matrix profile")
  
  

  fig.update_yaxes(range=[test_df["Close"].min(), test_df["Close"].max()], showgrid=True)
  fig.update_xaxes(showgrid=True)

  

  

  fig.show()

  fig1 = go.Figure()

  '''fig1.add_trace(go.Bar(
    x = vwap_df.index,
    y = scores,
    name='scores',
    
))'''
  fig1.add_trace(go.Scatter(x=test_df.index, y=scores,
                    mode='lines',
                    name='close'),
              )

  fig1.show()

In [ ]:
def generate_signal(arr, patterns_series, treshold):
  global state
  
  distance_profile = stumpy.mass(arr["otn"], patterns_series["otn"])
  dist = np.min(distance_profile)
  


  if dist > treshold:
    state = 0

  if dist < treshold and arr["Close"].iloc[-1] > arr["+band"].iloc[-1]:
    state = -1

  if dist < treshold and arr["Close"].iloc[-1] < arr["-band"].iloc[-1]:
    state = 1

  if state == -1 and arr["Close"].iloc[-1] <= arr["WVAP"].iloc[-1]:
    state = 0

  if state == 1 and arr["Close"].iloc[-1] >= arr["WVAP"].iloc[-1]:
    state = 0
  
  return dist

In [ ]:
def generate_signal1(arr, patterns_series, treshold):
  
  global direction
  distance_profile = stumpy.mass(arr["otn"], patterns_series["otn"])
  dist = np.min(distance_profile)
  
  
  

  if dist < treshold and arr["Close"].iloc[-1] >= arr["+band"].iloc[-1]:
    
      direction =-1

  if dist < treshold and arr["Close"].iloc[-1] < arr["-band"].iloc[-1]:
     direction = 1
      

  if direction == -1 and arr["Close"].iloc[-1] < arr["WVAP"].iloc[-1]:
     
    direction = 0

  if direction == 1 and arr["Close"].iloc[-1] > arr["WVAP"].iloc[-1]:
    
    direction = 0

  if arr["+band"].iloc[-1] ==  arr["-band"].iloc[-1]:

    direction = 0




  

In [ ]:
def min_dist_value(patterns, check_patterns, test_pattern_size):
  mp = stumpy.stump(T_A = patterns["otn"].astype(float),
                        m = test_pattern_size,
                        T_B = check_patterns["otn"].astype(float).astype(float),
                        ignore_trivial = False)
  return np.min(mp[:,[0]])

In [ ]:
df=pd.read_csv(io.BytesIO(uploaded['LABU1-Minute-Trade.csv']) ,parse_dates=[['Date', 'Time']])
df = df.loc[:, ["Date_Time", "Open","High", "Low", "Close", "TotalVolume"]]
train_df = df.iloc[: int(df.shape[0] * 0.6), :]
test_df = df.loc[int(df.shape[0] * 0.6) :, : ]
dates = test_df["Date_Time"].values
test_df = test_df.reset_index(drop=True)
train_df = add_VWAP_bands(train_df)
test_df = add_VWAP_bands(test_df)
test_df = test_df[:20000]
test_df = test_df.reset_index(drop=True)
pattern_size = 100
test_pattern_size = 10
min_dist_trash = 0.5  #100 good

<ipython-input-4-0ca723fb5eb1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date_Time'] = pd.to_datetime(df['Date_Time'])


In [ ]:
patterns = get_patterns(train_df[:train_df.shape[0]//2], pattern_size)
check_patterns = get_patterns(train_df[train_df.shape[0]//2:], pattern_size)
patterns["otn"] = patterns["Close"] / patterns["WVAP"]
test_df["otn"] = test_df["Close"] / test_df["WVAP"]
check_patterns["otn"] = check_patterns["Close"] / check_patterns["WVAP"]

<ipython-input-6-eddf66c3d1cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date_Time'] = pd.to_datetime(df['Date_Time'])
<ipython-input-6-eddf66c3d1cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date_Time'] = pd.to_datetime(df['Date_Time'])


In [ ]:
m_dist = min_dist_value(patterns, check_patterns, test_pattern_size)

In [ ]:
m_dist

0.1978935866967611

In [ ]:
signals = []
distances = []
pattern = 0
direction =0
for i in range(test_df.shape[0]):
  if i - pattern_size >=0:
    generate_signal1(test_df.iloc[i - test_pattern_size:i], patterns, 0.3)
    signals.append(direction)
    '''if dist < 2.8:
     distances.append(1)
    else:
      distances.append(0)'''

    

In [ ]:
signals.append(0)

In [ ]:
test_df = test_df[pattern_size -1 : ]
test_df.reset_index(drop=True)

,Open,High,Low,Close,TotalVolume,WVAP,+band,-band,Date_Time,otn
0,7.58,7.61,7.57,7.60,92601,7.754134,7.846081,7.662186,2022-09-15 18:08:00,0.980122
1,7.60,7.62,7.60,7.61,126020,7.753271,7.845627,7.660914,2022-09-15 18:09:00,0.981521
2,7.61,7.61,7.59,7.61,60296,7.752854,7.845413,7.660296,2022-09-15 18:10:00,0.981574
3,7.60,7.64,7.60,7.63,126382,7.752062,7.844922,7.659202,2022-09-15 18:11:00,0.984254
4,7.64,7.65,7.62,7.65,126045,7.751412,7.844390,7.658434,2022-09-15 18:12:00,0.986917
...,...,...,...,...,...,...,...,...,...,...
19256,6.74,6.74,6.71,6.73,93540,6.513129,6.640858,6.385401,2022-11-23 20:57:00,1.033297
19257,6.73,6.75,6.71,6.74,64542,6.513392,6.641269,6.385515,2022-11-23 20:58:00,1.034791
19258,6.73,6.75,6.72,6.75,144231,6.513991,6.642218,6.385763,2022-11-23 20:59:00,1.036231
19259,6.75,6.78,6.73,6.74,289370,6.515248,6.644276,6.386220,2022-11-23 21:00:00,1.034496


In [ ]:
test_df["Signal"] = signals
test_df = test_df.reset_index(drop=True)

In [ ]:
sell_index =np.where(test_df["Signal"].values == -1)
sell_index=np.array(sell_index[0])
buy_index =np.where(test_df["Signal"].values == 1)
buy_index=np.array(buy_index[0])


In [ ]:
import plotly.graph_objects as go



fig = go.Figure()
fig.add_trace(go.Scatter(x=test_df.index, y=test_df["Close"],
                    mode='lines',
                    name='close'))


fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df['WVAP'],
    mode='lines',
    name='VWAP', 
    line=dict(color='coral',width=2)
))
fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["+band"],
    mode='lines',
    name='VWAP + 1std', 
    line=dict(color='green',width=2)
))

fig.add_trace(go.Scatter(
    x=test_df.index,
    y=test_df["-band"],
    mode='lines',
    name='VWAP - 1std', 
    line=dict(color='green',width=2)

))

fig.add_trace(go.Bar(
    x=buy_index,
    y=[test_df["Close"].max() for i in range(len(buy_index))],
    name='up',
    marker_color='green', opacity=0.3
))
fig.add_trace(go.Bar(
    x=sell_index,
    y=[test_df["Close"].max() for i in range(len(sell_index))],
    name='down',
    marker_color='red',opacity=0.3
))


fig.update_layout(
    title="Размеченное движение цены на форварде с преобразованием log > корень 3-степени")
fig.update_yaxes(range=[test_df["Close"].min(), test_df["Close"].max()])

fig.show()

In [ ]:
test_df

,Open,High,Low,Close,TotalVolume,WVAP,+band,-band,Date_Time,otn,Signal
0,7.58,7.61,7.57,7.60,92601,7.754134,7.846081,7.662186,2022-09-15 18:08:00,0.980122,0
1,7.60,7.62,7.60,7.61,126020,7.753271,7.845627,7.660914,2022-09-15 18:09:00,0.981521,0
2,7.61,7.61,7.59,7.61,60296,7.752854,7.845413,7.660296,2022-09-15 18:10:00,0.981574,0
3,7.60,7.64,7.60,7.63,126382,7.752062,7.844922,7.659202,2022-09-15 18:11:00,0.984254,0
4,7.64,7.65,7.62,7.65,126045,7.751412,7.844390,7.658434,2022-09-15 18:12:00,0.986917,0
...,...,...,...,...,...,...,...,...,...,...,...
19256,6.74,6.74,6.71,6.73,93540,6.513129,6.640858,6.385401,2022-11-23 20:57:00,1.033297,-1
19257,6.73,6.75,6.71,6.74,64542,6.513392,6.641269,6.385515,2022-11-23 20:58:00,1.034791,-1
19258,6.73,6.75,6.72,6.75,144231,6.513991,6.642218,6.385763,2022-11-23 20:59:00,1.036231,-1
19259,6.75,6.78,6.73,6.74,289370,6.515248,6.644276,6.386220,2022-11-23 21:00:00,1.034496,-1


In [ ]:
save_df = test_df[["Open","High","Low","Close"]]

In [ ]:
save_df["Volume"] = test_df["TotalVolume"].values
save_df["Signal"] = test_df["Signal"].values
save_df["Datetime"] = dates[pattern_size -1:]

<ipython-input-240-b2836cb83195>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-240-b2836cb83195>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-240-b2836cb83195>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
dates[0]

numpy.datetime64('2022-09-15T16:29:00.000000000')

In [ ]:
save_df

,Open,High,Low,Close,Volume,Signal,Datetime
0,7.58,7.61,7.57,7.60,92601,0,2022-09-15 18:08:00
1,7.60,7.62,7.60,7.61,126020,0,2022-09-15 18:09:00
2,7.61,7.61,7.59,7.61,60296,0,2022-09-15 18:10:00
3,7.60,7.64,7.60,7.63,126382,0,2022-09-15 18:11:00
4,7.64,7.65,7.62,7.65,126045,0,2022-09-15 18:12:00
...,...,...,...,...,...,...,...
19256,6.74,6.74,6.71,6.73,93540,-1,2022-11-23 20:57:00
19257,6.73,6.75,6.71,6.74,64542,-1,2022-11-23 20:58:00
19258,6.73,6.75,6.72,6.75,144231,-1,2022-11-23 20:59:00
19259,6.75,6.78,6.73,6.74,289370,-1,2022-11-23 21:00:00


In [ ]:
save_df.to_csv("Antropos_test.csv")